In [5]:
import sympy
from gekko import GEKKO

input_strings = ["(x - y)**0.5 + x - 1", "sin(x - y) + x*y - 1*exp(2)"]

m = GEKKO()

# Define variables
x = m.Var()
y = m.Var()

equations = []
for input_string in input_strings:
    equation_expression = sympy.parse_expr(input_string)
    equation_lambda = sympy.lambdify((x, y), equation_expression)
    equation = m.Equation(equation_lambda(x.value, y.value))
    equations.append(equation)

m.Equations(equations)

TypeError: 'int' object is not subscriptable

apm 103.27.8.106_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           25
   Intermediates:            0
   Connections  :            0
   Equations    :           25
   Residuals    :           25
 
 Number of state variables:             25
 Number of total equations: -           25
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co